# Compressing and Cleaning Datasets

## Song data from Genius

Compressed the genius song data into only english songs, a range from 1980 to 2019, and views greater than or equal to 10000 and cleaned the lyrical column

In [ ]:
import pandas as pd

cols = ['title', 'tag', 'artist', 'year', 'views','lyrics', 'language_cld3']


songs_df = pd.read_csv('../data/raw/song_lyrics.csv', usecols=cols, chunksize=100000)

filtered = []

for data in songs_df:
    data = data[(data['language_cld3'] == 'en') & ((data['year'] >= 1980) & (data['year'] <= 2019)) & (data['views'] >= 10000)]

    data['lyrics'] = data['lyrics'].fillna('').astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    filtered.append(data)

df = pd.concat(filtered, ignore_index=True)

songs_compressed = df.groupby('year').apply(lambda x: x.sample(n=min(len(x), 275), random_state=42, replace=False)).reset_index(drop=True)

songs_compressed.to_csv('../data/processed/songs_compressed_2019.csv', index=False)

## Billboard data